In [65]:
import pandas as pd
import numpy as np
from datetime import datetime

In [66]:
df = pd.read_csv("data/loan_data_2007_2014.csv")

/var/folders/2h/rjm2fb_j53gb0nswwgx_xyfc0000gn/T/ipykernel_20077/734496051.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datasets/loan_data_2007_2014.csv")


In [67]:
df.columns

Index(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url',
       'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verificatio

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 75 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   466285 non-null  int64  
 1   id                           466285 non-null  int64  
 2   member_id                    466285 non-null  int64  
 3   loan_amnt                    466285 non-null  int64  
 4   funded_amnt                  466285 non-null  int64  
 5   funded_amnt_inv              466285 non-null  float64
 6   term                         466285 non-null  object 
 7   int_rate                     466285 non-null  float64
 8   installment                  466285 non-null  float64
 9   grade                        466285 non-null  object 
 10  sub_grade                    466285 non-null  object 
 11  emp_title                    438697 non-null  object 
 12  emp_length                   445277 non-null  object 
 13 

# 1. Preprocess Continuos variables

## Employment Length

In [70]:
df['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [71]:
df['emp_length_int'] = df['emp_length'].map({'10+ years':10,'< 1 year':0,'1 year':1,'3 years':3,'8 years':8,'9 years':9,'4 years':4,'5 years':5,
                     '6 years':6,'2 years':2,'7 years':7,'nan':0})

In [48]:
df.shape

(466285, 76)

In [10]:
type(df['emp_length_int'][0])

numpy.float64

## Earliest credit line date

In [72]:
df['earliest_cr_line_date'] = pd.to_datetime(df['earliest_cr_line'],format='%b-%y')

In [73]:

# Convert 'earliest_cr_line_date' to datetime format
df['earliest_cr_line_date'] = pd.to_datetime(df['earliest_cr_line_date'])
# Calculate the number of months since the earliest credit line date
current_date = datetime.now()
df['months_since_earliest_cr_line'] = (current_date - df['earliest_cr_line_date']).apply(lambda x: x.days // 30)-40



In [74]:
df.loc[:,['earliest_cr_line','earliest_cr_line_date','months_since_earliest_cr_line']][df['months_since_earliest_cr_line']<0]

,earliest_cr_line,earliest_cr_line_date,months_since_earliest_cr_line
1580,Sep-62,2062-09-01,-506.0
1770,Sep-68,2068-09-01,-579.0
2799,Sep-64,2064-09-01,-530.0
3282,Sep-67,2067-09-01,-567.0
3359,Feb-65,2065-02-01,-535.0
...,...,...,...
464003,Jan-68,2068-01-01,-571.0
464260,Jul-66,2066-07-01,-552.0
465100,Oct-67,2067-10-01,-568.0
465500,Sep-67,2067-09-01,-567.0


In [75]:
df['months_since_earliest_cr_line'][df['months_since_earliest_cr_line']<0] = df['months_since_earliest_cr_line'].max()

/var/folders/2h/rjm2fb_j53gb0nswwgx_xyfc0000gn/T/ipykernel_20077/1837468098.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['months_since_earliest_cr_line'][df['months_since_earliest_cr_line']<0] = df['months_since_earliest_cr_line'].

In [76]:
df['months_since_earliest_cr_line'].describe()
# Shows some descriptive statisics for the values of a column.
# Dates from 1969 and before are not being converted well, i.e., they have become 2069 and similar,
# and negative differences are being calculated.

count    466256.000000
mean        284.910309
std          87.724506
min         113.000000
25%         226.000000
50%         268.000000
75%         328.000000
max         635.000000
Name: months_since_earliest_cr_line, dtype: float64

## Term

In [55]:
df.shape

(466285, 78)

In [77]:
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [78]:
df['term_int'] = df['term'].map({' 36 months' : 36, ' 60 months': 60})

In [79]:
df['term_int'].unique

<bound method Series.unique of 0         36
1         60
2         36
3         36
4         60
          ..
466280    60
466281    60
466282    60
466283    36
466284    36
Name: term_int, Length: 466285, dtype: int64>

## Issue date

In [80]:
df['issue_d']

0         Dec-11
1         Dec-11
2         Dec-11
3         Dec-11
4         Dec-11
           ...  
466280    Jan-14
466281    Jan-14
466282    Jan-14
466283    Jan-14
466284    Jan-14
Name: issue_d, Length: 466285, dtype: object

In [81]:
df['issue_d_dates'] = pd.to_datetime(df['issue_d'],format='%b-%y')

In [82]:
df['issue_d_dates']

0        2011-12-01
1        2011-12-01
2        2011-12-01
3        2011-12-01
4        2011-12-01
            ...    
466280   2014-01-01
466281   2014-01-01
466282   2014-01-01
466283   2014-01-01
466284   2014-01-01
Name: issue_d_dates, Length: 466285, dtype: datetime64[ns]

In [83]:
df.shape

(466285, 80)

In [84]:
df['mths_since_issue_d'] = (current_date - df['issue_d_dates']).apply(lambda x: x.days // 30)-40

In [85]:
df['mths_since_issue_d']

0         112
1         112
2         112
3         112
4         112
         ... 
466280     87
466281     87
466282     87
466283     87
466284     87
Name: mths_since_issue_d, Length: 466285, dtype: int64

In [86]:
df.shape

(466285, 81)

# 2. Preprocess Discrete variables

In [24]:
loan_data_dummies = [pd.get_dummies(df['grade'], prefix = 'grade', prefix_sep = ':'),
                     pd.get_dummies(df['sub_grade'], prefix = 'sub_grade', prefix_sep = ':'),
                     pd.get_dummies(df['home_ownership'], prefix = 'home_ownership', prefix_sep = ':'),
                     pd.get_dummies(df['verification_status'], prefix = 'verification_status', prefix_sep = ':'),
                     pd.get_dummies(df['loan_status'], prefix = 'loan_status', prefix_sep = ':'),
                     pd.get_dummies(df['purpose'], prefix = 'purpose', prefix_sep = ':'),
                     pd.get_dummies(df['addr_state'], prefix = 'addr_state', prefix_sep = ':'),
                     pd.get_dummies(df['initial_list_status'], prefix = 'initial_list_status', prefix_sep = ':')]

In [26]:
loan_data_dummies = pd.concat(loan_data_dummies, axis = 1)

In [27]:
df = pd.concat([df,loan_data_dummies],axis = 1)

In [28]:
df.isnull().sum()

Unnamed: 0               0
id                       0
member_id                0
loan_amnt                0
funded_amnt              0
                        ..
addr_state:WI            0
addr_state:WV            0
addr_state:WY            0
initial_list_status:f    0
initial_list_status:w    0
Length: 207, dtype: int64

In [29]:
df['total_rev_hi_lim'].isnull().sum()

70276

Total revenue limit

In [30]:
df['total_rev_hi_lim'].fillna(df['funded_amnt'], inplace=True)

/var/folders/2h/rjm2fb_j53gb0nswwgx_xyfc0000gn/T/ipykernel_20077/2587915171.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['total_rev_hi_lim'].fillna(df['funded_amnt'], inplace=True)


In [31]:
df['total_rev_hi_lim'].isnull()

0         False
1         False
2         False
3         False
4         False
          ...  
466280    False
466281    False
466282    False
466283    False
466284    False
Name: total_rev_hi_lim, Length: 466285, dtype: bool

In [32]:
df['annual_inc'].fillna(df['annual_inc'].mean(),inplace = True)

/var/folders/2h/rjm2fb_j53gb0nswwgx_xyfc0000gn/T/ipykernel_20077/3700803366.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['annual_inc'].fillna(df['annual_inc'].mean(),inplace = True)


‘mths_since_earliest_cr_line’

‘acc_now_delinq’

‘total_acc’

‘pub_rec’

‘open_acc’

‘inq_last_6mths’

‘delinq_2yrs’

‘emp_length_int’

In [33]:
columns_to_fill = [
    'months_since_earliest_cr_line',
    'acc_now_delinq',
    'total_acc',
    'pub_rec',
    'open_acc',
    'inq_last_6mths',
    'delinq_2yrs',
    'emp_length_int'
]

# Fill null values with 0 for the specified columns
df[columns_to_fill] = df[columns_to_fill].fillna(0)

In [34]:
df.to_csv('Preprocessed.csv',index =False)

In [ ]:
df['months_since_earliest_cr_line'].describe()

count    466285.000000
mean        284.892589
std          87.750547
min           0.000000
25%         226.000000
50%         268.000000
75%         328.000000
max         635.000000
Name: months_since_earliest_cr_line, dtype: float64

In [41]:
a = pd.read_csv("Preprocessed.csv")
a.shape

/var/folders/2h/rjm2fb_j53gb0nswwgx_xyfc0000gn/T/ipykernel_20077/3644017061.py:1: DtypeWarning: Columns (20,48) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv("Preprocessed.csv")


(466285, 207)